In [1]:
import tweepy
print(tweepy.__version__)

3.10.0


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
from tqdm import tqdm

In [37]:
subset1 = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Code\JH\data\0020공시11.csv',index_col=0)
subset2 = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Code\JH\data\0014공시1,3,4,14,15,19,24,27,63,65,67.csv',index_col=0)
subset3 = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Code\JH\data\1520공시1,3,4,14,15,19,24,27,63,65,67.csv',index_col=0)

In [49]:
train_data = pd.concat([subset1, subset2, subset3], axis=0)
train_data = train_data.reset_index()

In [50]:
train_data['text'] = train_data['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")
train_data

C:\Users\JH\AppData\Local\Temp/ipykernel_21968/1482995617.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['text'] = train_data['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")


,index,회사명,거래소코드,공시발생일,text,link
0,0,데코,013650,2002-12-31 15:11,대표이사등변경안내구분대표이사변경내용가변경전대표이사이재성대표이사김우영나변경후대표이사김...,https://kind.krx.co.kr/external/2002/12/31/K00...
1,1,다휘,055250,2002-12-30 14:18,대표이사등변경안내구분대표이사변경내용가변경전오석인나변경후오석인고지환각자대표이사변경이유...,https://kind.krx.co.kr/external/2002/12/30/K00...
2,2,이수페타시스,007660,2002-12-23 13:56,대표이사등변경안내구분대표이사변경내용가변경전김상범김종택각자대표이사나변경후김상범김종택김...,https://kind.krx.co.kr/external/2002/12/23/K00...
3,3,인지디스플레,037330,2002-12-14 14:00,정정공시년월일정정관련공시서류대표이사등변경안내정정관련공시서류제출일정정사유변경후대표이사...,https://kind.krx.co.kr/external/2002/12/14/K00...
4,4,인지디스플레,037330,2002-12-14 13:28,대표이사등변경안내구분대표이사변경내용가변경전임상호나변경후정구용이경주변경이유임상호대표이...,https://kind.krx.co.kr/external/2002/12/14/K00...
...,...,...,...,...,...,...
5306,423,매일홀딩스,005990,2018-01-31 17:37,해산사유발생해산사유발생자회사인엠즈푸드주식회사의주요경영사항신고해산사유자회사간합병해산내...,https://kind.krx.co.kr/external/2018/01/31/000...
5307,424,메디톡스,086900,2018-01-26 17:21,벌금등의부과정정신고보고정정일자정정관련공시서류벌금등의부과정정관련공시서류제출일정정사유조...,https://kind.krx.co.kr/external/2018/01/26/000...
5308,425,행남사,008800,2018-01-11 17:52,파산신청파산신청사건번호하합파산선고신청인회사와의관계농업회사법인주식회사엔트네이처팜채권자...,https://kind.krx.co.kr/external/2018/01/11/000...
5309,426,레이젠,047440,2018-01-04 18:30,파산신청기각파산신청기각사건번호하합기각일자관할법원대구지방법원기각사유법원의파산신청기각결...,https://kind.krx.co.kr/external/2018/01/04/000...


In [51]:
train_data['text'][3]

'정정공시년월일정정관련공시서류대표이사등변경안내정정관련공시서류제출일정정사유변경후대표이사의내용을자세히기재정정사항항목정정전정정후변경내용나변경후정구용이경주정구용이경주각자대표이사대표이사등변경안내구분대표이사변경내용가변경전임상호나변경후정구용이경주각자대표이사변경이유임상호대표이사해임변경일년월일기타'

In [52]:
stopwords = ['등','의','을','기타','년','월','일','공시']

In [53]:
# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Okt()

tokenized_data = []
for sentence in tqdm(train_data['text']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

 96%|█████████▌| 5084/5311 [3:20:50<05:14,  1.39s/it]

In [11]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [12]:
model.wv.vectors.shape

(3713, 100)

In [33]:
tokenized_data[:][0]

['대표이사',
 '변경',
 '안내',
 '1',
 '구분',
 '대표이사',
 '2',
 '변경',
 '내용',
 '가',
 '변경',
 '전',
 '대표이사',
 '이재성',
 '대표이사',
 '김우영',
 '나',
 '변경',
 '후',
 '대표이사',
 '김인식',
 '3',
 '변경',
 '이유',
 '대표이사',
 '사임',
 '에',
 '따르다',
 '신규',
 '선임',
 '4',
 '변경',
 '일',
 '2002년',
 '12월',
 '31일',
 '5']

In [35]:
print(model.wv.most_similar('강태우'))

[('김용호', 0.9004698991775513), ('배변', 0.8789714574813843), ('박희', 0.8769700527191162), ('가변', 0.8765009045600891), ('안종', 0.873893678188324), ('정', 0.8723775744438171), ('최상', 0.8696200847625732), ('윤종근', 0.8694169521331787), ('전이', 0.8666578531265259), ('권혁', 0.863219141960144)]
